In [2]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [3]:
openai = OpenAI()

In [4]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [5]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
web = Website("https://mevotech.applytojob.com/apply/FRt0RlwFAu?source=LinkedIn")
print(f"Title: {web.title}\n")
print(f"Text: {web.text}...")

Title: Artificial Intelligence Analyst - Mevotech - Career Page

Text: Skip To Job Description
View All Jobs
View Our Website
Artificial Intelligence Analyst
North York, ON, Canada
Full Time
IT
Experienced
Share
Take your Career to the next Level with MEVOTECH.
We are constantly challenging the status quo and raising the bar allowing employees to make an impact within the company.
Mevotech is a trusted North American aftermarket auto parts leader that specializes in the engineering, design and distribution of driveline, steering and suspension parts (branded and private label). Fueled by innovation, our brands feature engineering improvements for maximum durability and performance, as well as time and labor-saving enhancements that drive bay efficiencies and increase shop revenues. Our in-house engineering team always has the Professional Technician in mind, and we are continuously optimizing part design for durability and ease of installation.
With over 600 Canadian and U.S. team memb

In [7]:
system_prompt = """
You are a professional career advisor specializing in crafting tailored resumes.
Given a URL containing a job description, perform the following steps:
1. Retrieve and carefully analyze the job description from the provided URL.
2. Generate the following clearly structured resume sections based on the analysis:
Objective:
- 1–2 concise sentences summarizing the applicant's suitability for the role.
Highlights of Qualifications:
- Exactly 7 bullet points highlighting relevant experiences, achievements, and competencies aligned with the job description.
Technical Skills:
- A clear list of essential technical skills directly relevant to the job.

Make sure your entire response is formatted in Markdown for clarity and readability.
Do not wrap your response in code blocks or backticks - return the raw markdown content directly.
"""

In [8]:
def user_prompt():
    user_prompt = f"Youre looking at the job description website called {web.title}"
    user_prompt += f"\nHere is the job description text:{web.text}\n\n"
    return user_prompt

In [9]:
def message_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt()}
    ]

message_for(web)

[{'role': 'system',
  'content': "\nYou are a professional career advisor specializing in crafting tailored resumes.\nGiven a URL containing a job description, perform the following steps:\n1. Retrieve and carefully analyze the job description from the provided URL.\n2. Generate the following clearly structured resume sections based on the analysis:\nObjective:\n- 1–2 concise sentences summarizing the applicant's suitability for the role.\nHighlights of Qualifications:\n- Exactly 7 bullet points highlighting relevant experiences, achievements, and competencies aligned with the job description.\nTechnical Skills:\n- A clear list of essential technical skills directly relevant to the job.\n\nMake sure your entire response is formatted in Markdown for clarity and readability.\nDo not wrap your response in code blocks or backticks - return the raw markdown content directly.\n"},
 {'role': 'user',
  'content': 'Youre looking at the job description website called Artificial Intelligence Anal

In [10]:
def generate_resume_sections(job_description_url):
    """
    Generate resume sections based on the job description.
    """
    website = Website(job_description_url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=message_for(website),
        max_tokens=1500,
        temperature=0.5
    )
    return response.choices[0].message.content.strip()

def display_summary(url):
    summary = generate_resume_sections(url)
    display(Markdown(summary))

In [11]:
generate_resume_sections("https://mevotech.applytojob.com/apply/FRt0RlwFAu?source=LinkedIn")

'# Resume for Artificial Intelligence Analyst Position\n\n## Objective:\nDedicated and results-driven AI Analyst with a strong background in artificial intelligence and data science, seeking to leverage expertise in AI strategy development and implementation at Mevotech. Committed to enhancing operational efficiency and service delivery through innovative AI solutions.\n\n## Highlights of Qualifications:\n- Over 3 years of experience in AI strategy development, focusing on operational efficiency and service delivery improvement.\n- Proficient in conducting comprehensive needs assessments and data analysis to identify AI integration opportunities.\n- Strong knowledge of machine learning algorithms, neural networks, and deep learning techniques to design effective AI solutions.\n- Experienced in evaluating and recommending AI technologies and tools, including conducting cost-benefit analyses and risk assessments.\n- Proven ability to communicate complex AI concepts and findings to non-te

In [12]:
display_summary("https://mevotech.applytojob.com/apply/FRt0RlwFAu?source=LinkedIn")

# Resume for Artificial Intelligence Analyst Position

## Objective:
Dynamic and results-driven AI Analyst with a strong background in artificial intelligence and data science, seeking to leverage expertise in AI technologies and strategic planning to enhance operational efficiency at Mevotech.

## Highlights of Qualifications:
- Over 5 years of experience in AI strategy development, implementation, and evaluation, aligning with organizational goals.
- Proven track record in conducting comprehensive needs assessments to identify and integrate AI solutions effectively.
- Extensive knowledge of machine learning algorithms, neural networks, and deep learning techniques, ensuring innovative solutions.
- Proficient in programming languages such as Python and R, with hands-on experience in AI frameworks like TensorFlow and PyTorch.
- Demonstrated ability to communicate complex AI concepts to non-technical stakeholders, driving cross-functional collaboration.
- Skilled in project management, overseeing resource allocation and ensuring successful delivery of AI initiatives.
- Strong analytical skills with experience in cost-benefit analysis and risk assessment for informed decision-making on AI adoption.

## Technical Skills:
- **Programming Languages:** Python, R, Java
- **AI Frameworks:** TensorFlow, PyTorch, Keras
- **Cloud Platforms:** AWS, Azure, Google Cloud
- **Big Data Technologies:** Hadoop, Spark
- **Machine Learning Techniques:** Neural Networks, Deep Learning, LLM, Transformer Architecture
- **Data Analysis Tools:** Statistical Analysis, Data Visualization
- **Project Management:** Resource Allocation, Project Planning, Quality Assurance